In [1]:
print("🚀 Starting Data Transformation Pipeline")
print("=" * 50)

🚀 Starting Data Transformation Pipeline


# Data Transformation Pipeline for MLOps Using SageMaker Jupyter Notebooks
`This notebook demonstrates key data transformation techniques commonly used in machine learning pipelines. It follows MLOps best practices for data preprocessing and feature engineering using AWS SageMaker JupyterLab.`

## 1. Environment Setup and Configuration
### 📦 Step 1: Setup Environment

In [2]:
# 📦 Step 1: Setup Environment
import sagemaker
import boto3
import pandas as pd
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor
import json
import numpy as np
from datetime import datetime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

print(f"SageMaker Role: {role}")
print(f"Default Bucket: {bucket}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
SageMaker Role: arn:aws:iam::910316760829:role/service-role/AmazonSageMaker-ExecutionRole-20250720T171468
Default Bucket: sagemaker-us-east-1-910316760829


### ⚙️ Step 2: Data Generation
Creating a realistic dataset that simulates common data quality challenges found in production environments.

In [3]:
import pandas as pd
import numpy as np
import json
import random
from datetime import datetime, timedelta
import os

# Set random seed for reproducibility
np.random.seed(42)

# Number of records
num_records = 20000

# Generate random data
data = {
    "id": np.arange(1, num_records + 1),
    "name": [f"Name_{i}" for i in np.random.randint(1, 1000, num_records)],
    "age": np.random.randint(18, 80, num_records),
    "salary": np.random.choice([50000, 60000, 70000, None], num_records),
    "hire_date": [
        (datetime.now() - timedelta(days=random.randint(0, 3650))).strftime("%Y-%m-%d")
        if random.random() > 0.1 else None
        for _ in range(num_records)
    ],
    "profile": [
        json.dumps({
            "address": f"Street {random.randint(1, 100)}, City {random.randint(1, 50)}",
            "phone": f"{random.randint(1000000000, 9999999999)}",
            "email": f"email_{random.randint(1, 1000)}@example.com"
        })
        if random.random() > 0.1 else None
        for _ in range(num_records)
    ],
    "department": np.random.choice(["HR", "IT", "Finance", "Marketing", None], num_records),
    "bonus": [None if random.random() > 0.9 else random.randint(1000, 10000) for _ in range(num_records)]
}

# Create DataFrame
df = pd.DataFrame(data)

# Introduce some NaN values randomly
df.loc[np.random.choice(df.index, size=int(num_records * 0.05), replace=False), "age"] = np.nan
df.loc[np.random.choice(df.index, size=int(num_records * 0.1), replace=False), "salary"] = np.nan

# Ensure 'data' folder exists
os.makedirs("data", exist_ok=True)

# Save to CSV
df.to_csv("data/mock_data.csv", index=False)
print("Dataset created and uploaded to data/mock_data.csv")

Dataset created and uploaded to data/mock_data.csv


### ⚙️ Step 3: Upload Source Data to S3
Upload the source CSV dataset to input location in S3 (default bucket)  

In [4]:
s3 = boto3.resource('s3')
s3.meta.client.upload_file('data/mock_data.csv', bucket, 'input/mock_data.csv')
print(f"Dataset 'mock_data.csv' uploaded to: s3://{bucket}/input/mock_data.csv")

Dataset 'mock_data.csv' uploaded to: s3://sagemaker-us-east-1-910316760829/input/mock_data.csv


## 2. Data Exploration  
Load the raw dataset and perform initial data profiling. 
This step is crucial for understanding data quality and structure. 

### Step 1: Load the CSV File from S3 into the DataFrame

In [5]:
try:
    df = pd.read_csv(f's3://{bucket}/input/mock_data.csv')
    print(f"✅ Dataset loaded successfully!")
    print(f"📏 Dataset shape: {df.shape}")
except FileNotFoundError:
    print("❌ Error: mock_data.csv not found. Please run create_dataset.py first.")
    exit()

✅ Dataset loaded successfully!
📏 Dataset shape: (20000, 8)


### Step 2: Analyse the Data  
Perform comprehensive data analysis to understand:
- Data types and memory usage
- Missing values pattern
- Statistical distribution
- Unique values and categories

In [6]:
# Display the first 5 rows from the loaded DataFrame
print("\n📋 First 5 rows:")
df.head()


📋 First 5 rows:


,id,name,age,salary,hire_date,profile,department,bonus
0,1,Name_103,77.0,60000.0,2017-01-03,NaN,Marketing,6199.0
1,2,Name_436,62.0,50000.0,2017-12-15,"{""address"": ""Street 45, City 2"", ""phone"": ""162...",Marketing,8305.0
2,3,Name_861,61.0,60000.0,2021-02-15,"{""address"": ""Street 6, City 34"", ""phone"": ""558...",HR,3904.0
3,4,Name_271,36.0,70000.0,NaN,"{""address"": ""Street 99, City 20"", ""phone"": ""22...",NaN,6077.0
4,5,Name_107,78.0,60000.0,2017-06-09,"{""address"": ""Street 21, City 28"", ""phone"": ""18...",IT,9940.0


In [7]:
# Get the summary of the DataFrame
print("\n📊 Data Types & Non-Null Counts:\n")
df.info()


📊 Data Types & Non-Null Counts:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          20000 non-null  int64  
 1   name        20000 non-null  object 
 2   age         19000 non-null  float64
 3   salary      13519 non-null  float64
 4   hire_date   17988 non-null  object 
 5   profile     17980 non-null  object 
 6   department  16003 non-null  object 
 7   bonus       17993 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 1.2+ MB


In [8]:
# Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"\n🔄 Duplicate rows: {duplicates}")


🔄 Duplicate rows: 0


In [9]:
# Check unique values in the department column
df['department'].unique()

array(['Marketing', 'HR', nan, 'IT', 'Finance'], dtype=object)

In [10]:
# View statistical summary for numeric coloums
print("\n📈 Statistical Summary:")
df.describe(include='all')


📈 Statistical Summary:


,id,name,age,salary,hire_date,profile,department,bonus
count,20000.000000,20000,19000.000000,13519.000000,17988,17980,16003,17993.000000
unique,NaN,999,NaN,NaN,3622,17980,4,NaN
top,NaN,Name_825,NaN,NaN,2024-05-22,"{""address"": ""Street 68, City 10"", ""phone"": ""24...",IT,NaN
freq,NaN,37,NaN,NaN,14,1,4058,NaN
mean,10000.500000,NaN,48.444684,59962.275316,NaN,NaN,NaN,5480.194909
std,5773.647028,NaN,17.892848,8200.588356,NaN,NaN,NaN,2598.626609
min,1.000000,NaN,18.000000,50000.000000,NaN,NaN,NaN,1000.000000
25%,5000.750000,NaN,33.000000,50000.000000,NaN,NaN,NaN,3210.000000
50%,10000.500000,NaN,48.000000,60000.000000,NaN,NaN,NaN,5491.000000
75%,15000.250000,NaN,64.000000,70000.000000,NaN,NaN,NaN,7711.000000


In [11]:
# Check for missing values
print("\n❓ Missing Values Analysis:\n")
df.isnull().sum()


❓ Missing Values Analysis:



id               0
name             0
age           1000
salary        6481
hire_date     2012
profile       2020
department    3997
bonus         2007
dtype: int64

## 🧹 3. Data Cleaning & Quality Improvement

### Step 1: Handle Missing values of age, and salary
Handle missing values in age and salary columns using appropriate strategies:
- For age: Use median (robust to outliers)
- For salary: Use median (robust to outliers)

In [12]:
# Analyze missing patterns
print("\n📊 Missing Value Patterns:")
print("Missing Age values:")
print(df[df['age'].isnull()][['age', 'salary', 'department']])


📊 Missing Value Patterns:
Missing Age values:
       age   salary department
44     NaN  60000.0  Marketing
115    NaN  60000.0         IT
127    NaN      NaN  Marketing
147    NaN  60000.0         HR
164    NaN  70000.0         IT
...    ...      ...        ...
19872  NaN  60000.0         HR
19921  NaN      NaN         HR
19940  NaN  70000.0        NaN
19997  NaN  60000.0         IT
19998  NaN  60000.0  Marketing

[1000 rows x 3 columns]


In [13]:
print("Missing Salary values")
print(df[df['salary'].isnull()][['age', 'salary', 'department']])

Missing Salary values
        age  salary department
5      35.0     NaN         IT
11     61.0     NaN         IT
13     46.0     NaN        NaN
14     48.0     NaN         IT
15     61.0     NaN         HR
...     ...     ...        ...
19984  71.0     NaN        NaN
19988  72.0     NaN  Marketing
19992  60.0     NaN        NaN
19993  76.0     NaN  Marketing
19999  47.0     NaN        NaN

[6481 rows x 3 columns]


In [14]:
# Get the median values for age, and salary
age_median = df['age'].median()
salary_median = df['salary'].median()
print("Age Median", age_median)
print("Salary Median", salary_median)

Age Median 48.0
Salary Median 60000.0


In [15]:
# Fill missing values of age with age_median
df['age'] = df['age'].fillna(age_median)
# Fill missing values of salary with salary_median
df['salary'] = df['salary'].fillna(salary_median)

#### Age & Salary columns missing values are filled with the respective median

In [16]:
# Verify the Age & Salary data
df.head()
# Check for missing values
print("Missing values in each column")
df.isnull().sum()

Missing values in each column


id               0
name             0
age              0
salary           0
hire_date     2012
profile       2020
department    3997
bonus         2007
dtype: int64

### Step 2: Handle Missing values of Department
Handle missing values in categorical columns:
- For department: Use 'Unknown' category
- This preserves the information that the department was missing

In [17]:
print("Print the missing values for Department\n")
print("Missing Department Missing values")
print(df[df['department'].isnull()][['age', 'salary', 'department']])

Print the missing values for Department

Missing Department Missing values
        age   salary department
3      36.0  70000.0        NaN
13     46.0  60000.0        NaN
49     34.0  50000.0        NaN
53     33.0  60000.0        NaN
57     28.0  70000.0        NaN
...     ...      ...        ...
19973  50.0  60000.0        NaN
19975  29.0  60000.0        NaN
19984  71.0  60000.0        NaN
19992  60.0  60000.0        NaN
19999  47.0  60000.0        NaN

[3997 rows x 3 columns]


In [18]:
# Fill the missing values in department with 'Unknown'
df['department'] = df['department'].fillna('Unknown')

#### Department column missing values are filled with the respective median

In [19]:
# Verify the Age & Salary data
df.head()
# Check for missing values
print("Missing values in each column")
print(df.isnull().sum())
# Check unique values in the department column
df['department'].unique()

Missing values in each column
id               0
name             0
age              0
salary           0
hire_date     2012
profile       2020
department       0
bonus         2007
dtype: int64


array(['Marketing', 'HR', 'Unknown', 'IT', 'Finance'], dtype=object)

### Step 3: Parse and Extract Profile Information
Devide Profile Column into 3 different columns i.e., Address, Phone, Email   

Parse JSON profile data and extract structured information:
- Extract address, phone, and email into separate columns
- Handle malformed JSON gracefully
- Maintain data integrity during extraction

In [20]:
print("Top rows from profile column \n")
print(df['profile'].head())

# Find the first non-null value in the column
profile_first_value = df['profile'].dropna().iloc[0]
# Print its type
print("\nProfile column values current data type")
print(type(profile_first_value))

# If your 'profile' column already contains Python dictionaries, not JSON strings.
# You do not need to parse it with json.loads(). The data is ready to be used directly.

# Convert profile JSON strings into dictionaries
df['profile'] = df['profile'].apply(lambda x: json.loads(x) if pd.notnull(x) else {})

Top rows from profile column 

0                                                  NaN
1    {"address": "Street 45, City 2", "phone": "162...
2    {"address": "Street 6, City 34", "phone": "558...
3    {"address": "Street 99, City 20", "phone": "22...
4    {"address": "Street 21, City 28", "phone": "18...
Name: profile, dtype: object

Profile column values current data type
<class 'str'>


In [21]:
# Extract Address Field
print("Extract Address Field....\n")
# Create new 'address' column by extracting from 'profile' dictionaries
df['address'] = df['profile'].apply(lambda x: x.get('address', None))  # Returns None if no address key

print("Top rows from profile column \n")
print(df['profile'].head())
print("\nTop rows from newly created address column \n")
print(df['address'].head())


Extract Address Field....

Top rows from profile column 

0                                                   {}
1    {'address': 'Street 45, City 2', 'phone': '162...
2    {'address': 'Street 6, City 34', 'phone': '558...
3    {'address': 'Street 99, City 20', 'phone': '22...
4    {'address': 'Street 21, City 28', 'phone': '18...
Name: profile, dtype: object

Top rows from newly created address column 

0                  None
1     Street 45, City 2
2     Street 6, City 34
3    Street 99, City 20
4    Street 21, City 28
Name: address, dtype: object


In [22]:
# Extract Phone Field
print("Extract Phone Field....\n")
# Create new 'phone' column by extracting from 'profile' dictionaries
df['phone'] = df['profile'].apply(lambda x: x.get('phone', None))  # Returns None if no address key

print("Top rows from profile column \n")
print(df['profile'].head())
print("\nTop rows from newly created phone column \n")
print(df['phone'].head())


Extract Phone Field....

Top rows from profile column 

0                                                   {}
1    {'address': 'Street 45, City 2', 'phone': '162...
2    {'address': 'Street 6, City 34', 'phone': '558...
3    {'address': 'Street 99, City 20', 'phone': '22...
4    {'address': 'Street 21, City 28', 'phone': '18...
Name: profile, dtype: object

Top rows from newly created phone column 

0          None
1    1621025422
2    5589424988
3    2299788720
4    1833307230
Name: phone, dtype: object


In [23]:
# Extract Email Field
print("Extract Email Field....\n")
# Create new 'email' column by extracting from 'profile' dictionaries
df['email'] = df['profile'].apply(lambda x: x.get('email', None))  # Returns None if no address key

print("Top rows from profile column \n")
print(df['profile'].head())
print("\nTop rows from newly created email column \n")
print(df['email'].head())

print(f"\n✅ Profile fields extracted:")

Extract Email Field....

Top rows from profile column 

0                                                   {}
1    {'address': 'Street 45, City 2', 'phone': '162...
2    {'address': 'Street 6, City 34', 'phone': '558...
3    {'address': 'Street 99, City 20', 'phone': '22...
4    {'address': 'Street 21, City 28', 'phone': '18...
Name: profile, dtype: object

Top rows from newly created email column 

0                     None
1    email_599@example.com
2      email_7@example.com
3    email_349@example.com
4    email_951@example.com
Name: email, dtype: object

✅ Profile fields extracted:


In [24]:
# Now drop the profile column
print("\nColumns before dropping profile:")
print(df.columns.tolist())

# Without inplace=True (df remains unchanged)
cleaned_df = df.drop(columns=['profile'])

# With inplace=True (df is modified directly)
#df.drop(columns=['profile'], inplace=True)

print("\nColumns in new DataFrame after dropping profile:")
# print(df.columns.tolist())
print(cleaned_df.columns.tolist())


Columns before dropping profile:
['id', 'name', 'age', 'salary', 'hire_date', 'profile', 'department', 'bonus', 'address', 'phone', 'email']

Columns in new DataFrame after dropping profile:
['id', 'name', 'age', 'salary', 'hire_date', 'department', 'bonus', 'address', 'phone', 'email']


### Step 4: Save cleaned data into new CSV and upload it to S3

In [25]:
print("\n💾 Saving cleaned data to: 'data/cleaned_data.csv' ...")
cleaned_df.to_csv("data/cleaned_data.csv", index=False)
print("✅ Cleaned data saved to: 'data/cleaned_data.csv'")

print(f"\nUploading dataset to s3 bucket: {bucket}")
s3.meta.client.upload_file('data/cleaned_data.csv', bucket, 'output/cleaned_data.csv')
print(f"Dataset 'mock_data.csv' uploaded to: s3://{bucket}/output/cleaned_data.csv")


💾 Saving cleaned data to: 'data/cleaned_data.csv' ...
✅ Cleaned data saved to: 'data/cleaned_data.csv'

Uploading dataset to s3 bucket: sagemaker-us-east-1-910316760829
Dataset 'mock_data.csv' uploaded to: s3://sagemaker-us-east-1-910316760829/output/cleaned_data.csv


## 4. Data Transformation & Feature Engineering

### Step 1: Load the cleaned dataset into new DataFrame

In [26]:
transform_df = pd.read_csv(f's3://{bucket}/output/cleaned_data.csv')
transform_df.head()

,id,name,age,salary,hire_date,department,bonus,address,phone,email
0,1,Name_103,77.0,60000.0,2017-01-03,Marketing,6199.0,NaN,NaN,NaN
1,2,Name_436,62.0,50000.0,2017-12-15,Marketing,8305.0,"Street 45, City 2",1.621025e+09,email_599@example.com
2,3,Name_861,61.0,60000.0,2021-02-15,HR,3904.0,"Street 6, City 34",5.589425e+09,email_7@example.com
3,4,Name_271,36.0,70000.0,NaN,Unknown,6077.0,"Street 99, City 20",2.299789e+09,email_349@example.com
4,5,Name_107,78.0,60000.0,2017-06-09,IT,9940.0,"Street 21, City 28",1.833307e+09,email_951@example.com


### Step 2 : Feature Engineering - Address Length
Create address length feature for potential geographic analysis.

In [27]:
# Create a new column 'address_length' 
print("\n🔧 Creating Address Length Feature...")
transform_df['address_length'] = transform_df['address'].apply(lambda x: len(str(x)))
print("Address followed by Address Length columns")
transform_df[['address', 'address_length']].head()


🔧 Creating Address Length Feature...
Address followed by Address Length columns


,address,address_length
0,NaN,3
1,"Street 45, City 2",17
2,"Street 6, City 34",17
3,"Street 99, City 20",18
4,"Street 21, City 28",18


### Step 3: Feature Engineering - Salary Categorization
Create salary categories for easier analysis and modeling.  
This converts continuous salary into ordinal categories.

In [28]:
print("\n🔧 Creating Salary Categories...")
# Define the bins and labels
bins = [0, 50000, 70000, 100000]
labels = ['low', 'medium', 'high']

# Create a new column 'salary_category'
transform_df['salary_category'] = pd.cut(df['salary'], bins=bins, labels=labels, include_lowest=True)

# Print sample data after adding the 'salary_category' column
print("Sample data after adding the 'salary_category' column: \n")
transform_df[['salary', 'salary_category']].head()


🔧 Creating Salary Categories...
Sample data after adding the 'salary_category' column: 



,salary,salary_category
0,60000.0,medium
1,50000.0,low
2,60000.0,medium
3,70000.0,medium
4,60000.0,medium


### Step 4: Feature Engineering - Age Groups  
Create age groups for demographic analysis.  
This helps in understanding age-based patterns in the data.  

In [29]:
print("\n🔧 Creating Age Groups...")
# Define age bins and labels
age_bins = [0, 25, 35, 45, 55, float('inf')]
age_labels = ['Young', 'Early Career', 'Mid Career', 'Senior', 'Experienced']

# Create a new column 'salary_category'
transform_df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, include_lowest=True)

# Age group distribution
print(f"Age group distribution:")
print(transform_df['age_group'].value_counts())

# Print sample data after adding the 'salary_category' column
print("\nSample data after adding the 'age_group' column: \n")
transform_df[['age', 'age_group']].head()



🔧 Creating Age Groups...
Age group distribution:
age_group
Experienced     7318
Senior          4068
Early Career    3142
Mid Career      3022
Young           2450
Name: count, dtype: int64

Sample data after adding the 'age_group' column: 



,age,age_group
0,77.0,Experienced
1,62.0,Experienced
2,61.0,Experienced
3,36.0,Mid Career
4,78.0,Experienced


### Step 5: Aggregation Features - Department Statistics  
Create department-level aggregations for comparative analysis.  
This enables understanding of department-wise patterns.  

In [30]:
print("\n🔧 Creating Department Statistics...")
# Group by 'department' and calculate average salary and age
department_summary_report = df.groupby('department').agg({
    'salary': 'mean',
    'age': 'mean'
}).reset_index()

# rename columns of department_summary_report for clarity
department_summary_report.columns = ['Department', 'Average Salary', 'Average Age']


🔧 Creating Department Statistics...


In [31]:
# Print the Summary Report
print("Summary report of average salary and age based on the department:\n")
print(department_summary_report)

Summary report of average salary and age based on the department:

  Department  Average Salary  Average Age
0    Finance    59830.035515    48.345256
1         HR    60015.155342    48.620106
2         IT    60034.499754    48.650074
3  Marketing    60049.455984    48.419139
4    Unknown    59939.954966    48.075056


### Step 6: Data Quality Metrics
Calculate data quality metrics for monitoring and MLOps.  
These metrics help track data drift and quality over time.  


In [32]:

print("\n📊 Data Quality Metrics...")

quality_metrics = {
    'total_rows': len(transform_df),
    'total_columns': len(transform_df.columns),
    'missing_values_count': transform_df.isnull().sum().sum(),
    'duplicate_rows': transform_df.duplicated().sum(),
    'numeric_columns': len(transform_df.select_dtypes(include=[np.number]).columns),
    'categorical_columns': len(transform_df.select_dtypes(include=['object']).columns),
    'unique_departments': transform_df['department'].nunique(),
    'unique_age_groups': transform_df['age_group'].nunique(),
    'unique_salary_categories': transform_df['salary_category'].nunique(),
    'processing_timestamp': datetime.now().isoformat()
}

print("Data Quality Metrics:")
for metric, value in quality_metrics.items():
    print(f"  {metric}: {value}")


📊 Data Quality Metrics...
Data Quality Metrics:
  total_rows: 20000
  total_columns: 13
  missing_values_count: 10079
  duplicate_rows: 0
  numeric_columns: 6
  categorical_columns: 5
  unique_departments: 5
  unique_age_groups: 5
  unique_salary_categories: 2
  processing_timestamp: 2025-07-21T00:16:54.071555


### Step 7: Save the transformed DataFrame to a new csv file

In [33]:
print("Saving Transformed data csv to: 'data/transformed_data.csv' ...")
transform_df.to_csv("data/transformed_data.csv", index=False)
print("\nTransformed data csv saved to: 'data/transformed_data.csv'")

s3.meta.client.upload_file('data/transformed_data.csv', bucket, 'output/transformed_data.csv')
print(f"Transformed data 'transformed_data.csv' uploaded to: s3://{bucket}/output/transformed_data.csv")

Saving Transformed data csv to: 'data/transformed_data.csv' ...

Transformed data csv saved to: 'data/transformed_data.csv'
Transformed data 'transformed_data.csv' uploaded to: s3://sagemaker-us-east-1-910316760829/output/transformed_data.csv


In [34]:
### Step 2: Save Department Statistics
print("Saving department statistics...")
department_summary_report.to_csv("data/department_statistics.csv", index=False)
print("✅ Department statistics saved to: 'data/department_statistics.csv'")

s3.meta.client.upload_file('data/department_statistics.csv', bucket, 'output/department_statistics.csv')
print(f"Department Statistics 'department_statistics.csv' uploaded to: s3://{bucket}/output/department_statistics.csv")

Saving department statistics...
✅ Department statistics saved to: 'data/department_statistics.csv'
Department Statistics 'department_statistics.csv' uploaded to: s3://sagemaker-us-east-1-910316760829/output/department_statistics.csv


## 5. Next Steps for MLOps

In [35]:
print(f"\n🎯 Next Steps for MLOps:")
print(f"  1. Model training using transformed features")
print(f"  2. Model validation and testing")
print(f"  3. Model deployment and monitoring")
print(f"  4. Data drift monitoring using quality metrics")
print(f"  5. Pipeline automation and orchestration")

print("\n" + "="*50)
print("🎉 DATA TRANSFORMATION PIPELINE COMPLETE!")
print("="*50)


🎯 Next Steps for MLOps:
  1. Model training using transformed features
  2. Model validation and testing
  3. Model deployment and monitoring
  4. Data drift monitoring using quality metrics
  5. Pipeline automation and orchestration

🎉 DATA TRANSFORMATION PIPELINE COMPLETE!
